# Missing TSTAT data on gridFTP/Globus transfers

## Reported problem

The Panorama project ran a series of Globlus/gridFTP from the host stash.osgconnect.net to the NERSC DTN cluster composed of a dozen hosts. Two data sets were then used to correlate information about the flows:

  . TSTAT data collected on the NERSC hosts and sent to two ELK clusters, NERSC cluster. The Panorama cluster is used by the Panorama team and the NERSC cluster is used when developing this notebook.
  . Globus logs collected on stash.osgconnect.net. This notebook was developed after receiving a subset of this data set which contains user information and therefore, while being used in this notebook, it cannot be made public. 
  
Having access to both data sets is required in order to execute this notebook.


## Investigation summary and status

The investigation followed the following steps:

. Repeat the reported problem on the NERSC cluster. Problem was replicated, therefore, there are no specific issues related to the Panorama cluster. . Problem was reported using the ELK Kabana web portal. Repeated problem directly using the ELK REST API. Problem was replicated, so no Kabana related problem. . Recovered TSTAT logs directly from NERSC DTN. Verified that the missing TSTAT were present (existed) on the DTN's.

This leaves with the following possible source of the problem:

. the tstat-send agent running on the DTN drop data when parsing the TSTAT logs. . the tstat-send agend and/or the NERSC RabbitMQ exchanger drop or stop some of the messages sent by the tstat-send agents. . the ELK clusters drop/ignore the missing entries. That would happen on both cluster (likelyhood ?)

Normally, the most likely place to look at would be the tstat-agent. For instance, perhaps the agent misses entries that are received while it processes the log ?

However, it was observed on both clusters, using the web portal to query the database, that the systems warn of failing to read some shards. After talking to the people running and supporting the NERSC ELK cluster, it appears there are currently too many indexes in the TSTAT set (about 30 indexes per month). It is not sure yet that this is the cause of missing data and this is still being investigated. This problem would, of course, exist on both clusters.

## Notebook organization

This notebook is used for debuging this problem. It contains code that queries the ELK clusters, read the Globus logs, DTN logs, parses and pre-process data. It also includes code that correlate datasets and find the missing entries. This code and the comments that comes along is the reasoning that leads to the previous summary and status of the investigation.

Note that some of this code is using library available at https://github.com/esnet/workflow-databooks/blob/master/datasets/tstat-nersc/tstat-dtn.py

## Query the NERSC Cluster

In [6]:
import sys
sys.path.append('..')
from tstat_nersc import create_tunnel, kill_tunnel, esdb_connect

# Create SSH tunnel
tunnel = create_tunnel()
# Connect to ELK
esdb = esdb_connect()

# Globals
tstat_index = "dtn-tstat-2018*"

proxy host:········
password: ········


/anaconda/lib/python2.7/site-packages/elasticsearch/connection/http_urllib3.py:112: UserWarning: Connecting to cheshire.nersc.gov using SSL with verify_certs=False is insecure.
  'Connecting to %s using SSL with verify_certs=False is insecure.' % host)


Retrieve all records of transfers from stash.osgconnect.net:

In [20]:
query={
    "query":{
        "bool": {
            "must": [
                {"match": { "source": "stash.osgconnect.net" }}
            ]
        }
    },
    "sort": [
        { "start": "asc" }
    ]
}

results = esdb.search(index=tstat_index, body=query, size=500)
print "total results: ", results['hits']['total']
raw = results['hits']['hits']

total results:  301


Filters sensor_id (destination DTN at NERSC) and time. Localize time to PST.

In [21]:
import csv
from pytz import timezone
import pytz
import datetime

def process_raw(raw):
    res={} 
    pst = pytz.timezone('America/Los_Angeles')
    for row in raw:
        dtn = row['_source']['meta']['sensor_id']
        if not dtn in res:
            res[dtn] = {'start':[],'end':[],'size':[]}
        start = datetime.datetime.fromtimestamp(int(row['_source']['start']))
        pst_start = pst.localize(start, is_dst=None)
        pst_start.astimezone (pst)
        res[dtn]['start'].append(pst_start)
        end = datetime.datetime.fromtimestamp(int(row['_source']['end']))
        pst_end = pst.localize(end, is_dst=None)
        pst_end.astimezone (pst)
        res[dtn]['end'].append(pst_end)
        res[dtn]['size'].append(int(row['_source']['file_size_MB']))
    return res 
        

In [22]:
tstat = process_raw(raw)

## Load and pre-process Globus log

Import gridftp logs. Only retrieve relevant information.

Process data, localize time to GMT

In [23]:
!cd ~/Downloads/transfer_logs; grep dtn nersc_gridftp.log  > /tmp/globus.csv
!cd ~/Downloads/transfer_logs/globus_logs; grep mbps */* > /tmp/mbps.csv
import csv
from pytz import timezone
import pytz

def read_extra():
    res = {'mbps':[],'size':[],'duration':[]}
    with open('/tmp/mbps.csv') as f:
        for row in csv.reader(f):
            res['mbps'].append(float(row[1].split(':')[1].lstrip()))
            res['duration'].append(row[2].split(':')[1][:-4].lstrip())
            res['size'].append(row[0].split(':')[-1].lstrip())
    return res

def read_globus():
    gmt = pytz.timezone('GMT')
    with open('/tmp/globus.csv') as f:
        res = {'time':[],'dtn':[],'streams':[]}
        for row in csv.reader(f):
            #t = datetime.datetime.strptime(row[3].split('.')[0], '%Y/%m/%d %H:%M:%S')
            t = datetime.datetime.strptime(row[10].split('.')[0], '%Y/%m/%d %H:%M:%S')
            gmt_t = gmt.localize(t, is_dst=None)
            gmt_t.astimezone (gmt)
            res['time'].append(gmt_t)
            res['dtn'].append(row[6])
            res['streams'].append(row[11])
        extra = read_extra()
        res.update(extra)
    return res


In [24]:
globus = read_globus() 

## Correlate both data sets to find missing TSTAT data

Detects streams that are in gridftp logs but not in tstat. The correlation is done by matching flows based on starting time. Since the TSTAT data is already filtered (by query) to flows coming from stash.osgconnect.net, we do not have to worry about other flows. A configurable "time proximity" in seconds can be set to set the time window for which we can consider that the TSTAT data relates to a given file transfer. The configurable is the optional "lim" option provided to check_tstat_transfer.

In [25]:
def check_tstat_transfer(tstat,dtn,transfer_time,lim=2):
    count = 0
    for t in tstat[dtn]['start']:
        if abs(transfer_time - t).seconds < lim:
            count += 1  
    return count
    
def check_transfers(globus, tstat):
    incorrect = {'time':[],'dtn':[],'shouldbe':[],'was':[],'mbps':[],'miss':[]}
    correct = {'time':[],'dtn':[],'shouldbe':[],'was':[],'mbps':[],'miss':[]}
    for t,dtn,mbps,streams in zip(globus['time'],globus['dtn'],globus['mbps'],globus['streams']):
        streams = int(streams)
        count = 0
        if dtn in tstat:
            count = check_tstat_transfer(tstat=tstat, dtn=dtn, transfer_time=t)
        res = None
        if count != streams:
            res = incorrect
        else:
            res = correct
        res['time'].append(t)
        res['dtn'].append(dtn)
        res['shouldbe'].append(streams)
        res['was'].append(count)
        res['mbps'].append(mbps)
        res['miss'].append(streams - count)
    return correct,incorrect
        
    

In [26]:
found,missing = check_transfers(globus=globus, tstat=tstat)
print "Number of missing streams: ", len(missing['time'])

Number of missing streams:  25


In [27]:
for dtn,t,mbps,miss in zip(missing['dtn'],missing['time'],missing['mbps'],missing['miss']):
    print dtn,t,mbps,miss

dtn01.nersc.gov 2018-02-28 08:30:12+00:00 309.98 4
dtn05.nersc.gov 2018-02-28 09:30:11+00:00 1589.78 4
dtn04.nersc.gov 2018-02-28 10:30:11+00:00 1274.91 4
dtn07.nersc.gov 2018-02-28 10:30:12+00:00 1965.45 4
dtn10.nersc.gov 2018-02-28 11:30:13+00:00 1799.97 4
dtn05.nersc.gov 2018-02-28 12:30:11+00:00 689.08 4
dtn02.nersc.gov 2018-02-28 12:30:12+00:00 1877.25 4
dtn05.nersc.gov 2018-02-28 13:00:12+00:00 1456.44 4
dtn05.nersc.gov 2018-02-28 13:30:12+00:00 1422.13 4
dtn05.nersc.gov 2018-02-28 14:00:11+00:00 1982.96 4
dtn04.nersc.gov 2018-02-28 14:30:14+00:00 1255.23 4
dtn09.nersc.gov 2018-02-28 15:30:13+00:00 1388.34 4
dtn01.nersc.gov 2018-02-28 15:30:14+00:00 1138.04 4
dtn09.nersc.gov 2018-02-28 16:30:11+00:00 1469.77 4
dtn04.nersc.gov 2018-02-28 17:30:10+00:00 1778.05 4
dtn05.nersc.gov 2018-02-28 18:00:12+00:00 1518.45 4
dtn08.nersc.gov 2018-02-28 18:30:13+00:00 1553.42 4
dtn10.nersc.gov 2018-02-28 18:30:12+00:00 1482.16 4
dtn01.nersc.gov 2018-02-28 19:00:11+00:00 1874.68 1
dtn04.nersc.go

From this result, we see we miss quite a few TSTAT entries. This confirms that the reported problem is indeed real and that there this is not a problem that happens only on the Panorama cluster since this was reproduced using the NERSC cluster.

## Looking at patterns in missing data

Very slow flows may be filtered by the tstat-agent running on the host. The following code looks at the missing TSTAT entries, and, after retrieve flow performance data from the Globus data, displays it.

In [28]:
for dtn,t,mbps,miss in zip(found['dtn'],found['time'],found['mbps'],found['miss']):
    print dtn,t,mbps,miss

dtn06.nersc.gov 2018-02-28 08:00:13+00:00 737.89 0
dtn06.nersc.gov 2018-02-28 09:00:11+00:00 1985.38 0
dtn09.nersc.gov 2018-02-28 09:00:12+00:00 1967.53 0
dtn06.nersc.gov 2018-02-28 09:30:12+00:00 1951.92 0
dtn08.nersc.gov 2018-02-28 10:00:11+00:00 816.95 0
dtn09.nersc.gov 2018-02-28 10:00:12+00:00 1306.28 0
dtn10.nersc.gov 2018-02-28 11:00:12+00:00 1613.12 0
dtn09.nersc.gov 2018-02-28 12:00:11+00:00 1545.36 0
dtn09.nersc.gov 2018-02-28 13:00:13+00:00 2003.69 0
dtn06.nersc.gov 2018-02-28 13:30:11+00:00 2009.21 0
dtn10.nersc.gov 2018-02-28 14:00:12+00:00 2033.28 0
dtn10.nersc.gov 2018-02-28 15:00:11+00:00 861.07 0
dtn10.nersc.gov 2018-02-28 16:00:11+00:00 1436.49 0
dtn06.nersc.gov 2018-02-28 16:00:12+00:00 1757.23 0
dtn06.nersc.gov 2018-02-28 16:30:12+00:00 1430.46 0
dtn01.nersc.gov 2018-02-28 17:00:13+00:00 1458.64 0
dtn07.nersc.gov 2018-02-28 17:00:14+00:00 1656.85 0
dtn06.nersc.gov 2018-02-28 17:30:11+00:00 1861.57 0
dtn10.nersc.gov 2018-02-28 18:00:12+00:00 2001.21 0
dtn06.nersc.gov

Some of the "missing" flows seem to have exceeded 400 Mbps. Since all flows were made of 4 different TCP streams, we can deduct that some of the missing flows were above 100 Mbps. In other words, we cannot just blames flows for being too slow for the configured filter.

Some other patterms is under investigation. For instance, flows were initiated every 30 minutes starting at the top of the hour. It is possible that the cron job running in the DTN hosts may ran the same time perhaps running into a bug in the agent.

## Checking TSTAT logs on NERSC DTN's

The next step is to verify if the TSTAT collection capture the missing flows. The NERSC DTN have been keeping all of its TSTAT measurement locally. After uploading the logs from dtn01.nersc.gov for the period of time when the problem was detecting, the following code parses the log and pre-processes it.

In [29]:
from pytz import timezone
import pytz

def read_tstat_from_dtn(filename):
    
    with open(filename) as f:
        pst = pytz.timezone('America/Los_Angeles')
        res = {'start':[],'end':[],'duration':[],'size':[],'speed':[]}
        for row in f.readlines():
            row = row.split(':')[1].split(' ')
            #t = datetime.datetime.strptime(row[3].split('.')[0], '%Y/%m/%d %H:%M:%S')
            #t = datetime.datetime.strptime(row[10].split('.')[0], '%Y/%m/%d %H:%M:%S')
            #gmt_t = gmt.localize(t, is_dst=None)
            #gmt_t.astimezone (gmt)
            res['start'].append(datetime.datetime.fromtimestamp(float(row[28])/1000,tz=pst))
            res['end'].append(datetime.datetime.fromtimestamp(float(row[29])/1000,tz=pst))
            res['duration'].append(float(row[30]))
            res['size'].append(int(row[6]))
            res['speed'].append(int(row[6]) / float(row[30]) / 1000)
    return res

tstat_dtn01 = read_tstat_from_dtn('/Users/lomax/data/tstat/dtn01.nersc.gov/2018-02-28')

The following displays flows, start and times, duration, data size, and speed from first, the local TSTAT logs (from DTN's) and second from the ELK cluster. A visual matching (sorry no code yet) concludes that the missing TSTAT logs are in the local logs but not in the ELK cluster.

In [30]:
for start, end, size, duration, speed in zip(tstat_dtn01['start'],tstat_dtn01['end'],tstat_dtn01['size'],tstat_dtn01['duration'],tstat_dtn01['speed']):
    print start,size/1024/1024, duration / 1000,int(speed * 8)

2018-02-28 00:30:12.404209-08:00 146 6.670935 184
2018-02-28 00:30:12.404227-08:00 211 6.670949 265
2018-02-28 00:30:12.404152-08:00 205 6.671045 258
2018-02-28 00:30:12.401261-08:00 209 6.673975 262
2018-02-28 07:30:14.132843-08:00 192 4.105387 394
2018-02-28 07:30:14.132850-08:00 191 4.105406 390
2018-02-28 07:30:14.132802-08:00 194 4.105501 397
2018-02-28 07:30:14.132563-08:00 195 4.105875 398
2018-02-28 09:00:13.550508-08:00 230 4.64077 416
2018-02-28 09:00:13.550547-08:00 129 4.640778 234
2018-02-28 09:00:13.550226-08:00 232 4.641122 419
2018-02-28 09:00:13.550528-08:00 180 4.64084 326
2018-02-28 11:00:11.592408-08:00 92 7.638757 101
2018-02-28 11:00:11.592266-08:00 169 7.638915 185
2018-02-28 11:00:11.592341-08:00 301 7.638902 330
2018-02-28 11:00:11.592046-08:00 210 7.63928 231
2018-02-28 11:30:11.252397-08:00 216 7.531838 240
2018-02-28 11:30:11.252106-08:00 172 7.532186 192
2018-02-28 11:30:11.252405-08:00 169 7.531933 189
2018-02-28 11:30:11.252381-08:00 214 7.531971 239
2018

In [31]:
for start,end,size in zip(tstat['dtn01.nersc.gov']['start'],tstat['dtn01.nersc.gov']['end'],tstat['dtn01.nersc.gov']['size']):
    print start,end,size, end - start, size * 8/(end - start).seconds

2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 133 0:00:10 106
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 133 0:00:10 106
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 250 0:00:10 200
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 131 0:00:10 104
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 250 0:00:10 200
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 131 0:00:10 104
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 294 0:00:10 235
2018-02-15 14:45:44-08:00 2018-02-15 14:45:54-08:00 294 0:00:10 235
2018-02-28 09:00:13-08:00 2018-02-28 09:00:18-08:00 243 0:00:05 388
2018-02-28 09:00:13-08:00 2018-02-28 09:00:18-08:00 136 0:00:05 217
2018-02-28 09:00:13-08:00 2018-02-28 09:00:18-08:00 189 0:00:05 302
2018-02-28 09:00:13-08:00 2018-02-28 09:00:18-08:00 241 0:00:05 385
2018-02-28 11:00:11-08:00 2018-02-28 11:00:19-08:00 177 0:00:08 177
2018-02-28 11:00:11-08:00 2018-02-28 11:00:19-08:00 220 0:00:08 220
2018-02-28 11:00:11-08:00 2018-02-28 11:00:19-08